In [ ]:
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
def extract_info(response_item):
    #название отзыва, полный текст отзыва и ссылка на него
    response_title = response_item.find("a", attrs={"class": "font-size-large font-bold"})
    response_name = response_title.text
    #print(response_name)
    
    link = response_title.get('href')
    response_url = "http://www.banki.ru{}".format(link)
    
    response = requests.get(response_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    response_text = soup.find("div",
                              attrs={"class": "article-text response-page__text markup-inside-small markup-inside-small--bullet"}
                             ).text.strip()
    
    #оценка и статус проверки
    response_rating = response_item.find("div", attrs={"class": "responses__item__rating"})
    is_grade = 'БЕЗ ОЦЕНКИ' in response_rating.text
    if is_grade == False:
        response_grade = response_item.find("span", attrs={"itemtype": "http://data-vocabulary.org/Rating"}).text.strip()
        problem_solved = 'ПРОБЛЕМА РЕШЕНА' in response_rating.text
    else:
        response_grade = None
        problem_solved = None
    
    #дата публикации
    response_time_html = response_item.find("time", attrs={"class": "display-inline-block"})
    if response_time_html != None:
        response_time = response_time_html.text
    else:
        response_time = None
    
    #название банка
    bank_name = response_item.find("a", attrs={"class": "responses__item__logo lazy-load"}).get('title')
    
    return{
        'response_name': response_name,
        'response_url': response_url,
        'response_text': response_text,
        'response_grade': response_grade,
        'problem_solved': problem_solved,
        'response_time': response_time,
        'bank_name': bank_name
    }

In [ ]:
%%time

is_countable = True  #учитывать только зачтенные отзывы?
max_sample_size = 5000  #размер выборки

sample_size = 0
page_num = 0
bank_responses_info = []

while True:
    page_num += 1
    if is_countable == True:
        url = "http://www.banki.ru/services/responses/list/?is_countable=on&page={}".format(page_num)
    else:
        url = "http://www.banki.ru/services/responses/list/?page={}".format(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for x in soup.find_all("article", attrs={"class": "responses__item"}):
        sample_size += 1
        bank_responses_info.append(extract_info(x))
    print ('page_num {}'.format(page_num), ' -> ', sample_size)
    if sample_size == max_sample_size:
        break

In [ ]:
with open('bank_responses_info.json', 'w') as f:
    json.dump(bank_responses_info, f)

In [ ]:
with open('100_responses_info.json', 'w') as f:
    json.dump(bank_responses_info[:100], f)